In [1]:
!pip install --upgrade pandas numpy dask black[jupyter] uproot3 pyarrow astropy

In [2]:
import uproot3

import dask.dataframe as dd
import pandas as pd
import numpy as np
from astropy import units as u
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
!ls -lah *.root

-rw-rw-r-- 1 jovyan users 76M May 24 22:29 data00.root
-rw-rw-r-- 1 jovyan users 76M May 24 22:29 data01.root
-rw-rw-r-- 1 jovyan users 76M May 24 22:29 data02.root
-rw-rw-r-- 1 jovyan users 76M May 24 22:29 data03.root
-rw-rw-r-- 1 jovyan users 76M May 24 22:29 data04.root
-rw-rw-r-- 1 jovyan users 76M May 24 22:30 data05.root
-rw-rw-r-- 1 jovyan users 76M May 24 22:30 data06.root
-rw-rw-r-- 1 jovyan users 76M May 24 22:30 data07.root
-rw-rw-r-- 1 jovyan users 76M May 24 22:30 data08.root
-rw-rw-r-- 1 jovyan users 76M May 24 22:30 data09.root
-rw-rw-r-- 1 jovyan users 76M May 24 22:30 data10.root
-rw-rw-r-- 1 jovyan users 76M May 24 22:30 data11.root
-rw-rw-r-- 1 jovyan users 76M May 24 22:30 data12.root
-rw-rw-r-- 1 jovyan users 76M May 24 22:30 data13.root
-rw-rw-r-- 1 jovyan users 76M May 24 22:30 data14.root
-rw-rw-r-- 1 jovyan users 76M May 24 22:30 data15.root
-rw-rw-r-- 1 jovyan users 76M May 24 22:31 data16.root
-rw-rw-r-- 1 jovyan users 76M May 24 22:31 data17.root
-rw-rw-r--

In [4]:
import scat

In [5]:
input_files = !ls data*.root

In [6]:
dask_dataframe = dd.from_map(scat.read_root_file, input_files)

In [7]:
#scat.dask_memory_usage_GB(dask_dataframe)

In [8]:
#centroids, histogram = scat.dask_histogram(dask_dataframe, "value1", bins=20, bins_range=[-5, 5])

In [9]:
#plt.bar(centroids, histogram)
#plt.grid();

In [10]:
class ComputeColumn:
    def __init__(self, column_1, column_2):
        self.column_1 = column_1
        self.column_2 = column_2
    def __call__(self, df):
        df = df.assign(computed_col=(df[self.column_1] + df[self.column_2]))**2
        return df

In [11]:
func = ComputeColumn("value1", "value2")

In [12]:
dask_dataframe_added_column = dask_dataframe.map_partitions(func)

In [13]:
class QueryFilter:
    def __init__(self, query):
        self.query = query
    def __call__(self, df):
        return df.query(self.query)

In [14]:
filt = QueryFilter("computed_col < 1")

In [15]:
dask_dataframe_added_column_filtered= dask_dataframe_added_column.map_partitions(filt)

In [16]:
output_pandas_dataframe= dask_dataframe_added_column_filtered.compute()

In [17]:
output_pandas_dataframe.head()

,seriesNumber,eventNumber,detNumber,value1,value2,computed_col
0,0,0,9,0.025411,0.161719,0.315340
1,0,0,4,1.186726,0.139076,0.513287
2,0,0,1,0.140486,1.308061,0.591192
3,0,0,1,0.319691,1.230156,0.295623
4,0,0,9,0.250476,0.729173,0.124920


In [18]:
len(output_pandas_dataframe) / 1e6

52.045638